# Moving Averages CINGULATE region

### 1) Imports

In [1]:
import os
import glob
import sys
import json

p = os.path.abspath('../')
if p not in sys.path:
    sys.path.append(p)

In [2]:
import moving_averages as ma
import colorado as cld
import dico_toolbox as dtx
from tqdm import tqdm

import pandas as pd
import numpy as np
from scipy.spatial import distance

import plotly.graph_objects as go
import pickle
import matplotlib.pyplot as plt

from soma import aims

import torch

from sklearn.cluster import KMeans, SpectralClustering, AffinityPropagation
from sklearn import metrics

import matplotlib.cm as cm

from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf, DictConfig

In [3]:
def closest_distance(centroid, df):
    """Returns closest point to centroid of a given cluster
    /!\ central_point is part of the defined set of points
    Args:
        IN: centroid: tuple of coordinates of centroid
            df: pandas.DataFrame with index corresponding to subjects and as many columns as coordinates
        OUT: central_point: subject ID corresponding to closest point to the centroid
    """
    # df must have as many columns as dimensions to consider
    distances = {}
    # Get distances to centroid for each point
    for sub in list(df.index):
        pos = [df.loc[sub][k] for k in range(1, len(df.columns)+1)]
        distances[sub] = distance.euclidean(pos, centroid)
    # Get closest point to centroid
    central_point = min(distances, key=distances.get)
    return central_point

In [4]:
path = '/neurospin/dico/data/deep_folding/current/crops/CINGULATE/mask/sulcus_based/2mm/centered_combined/hcp/Rcrops/'

### 2) Loading of subjects's distribution in the latent space
DataFrame is obtained during analysis of beta-VAE latent space (2 dimensions) for CINGULATE crops, according to the following steps:
- loading of trained model
- encoding of test controls and asymmetry benchmark subjects

In [5]:
if torch.cuda.is_available():
    device = "cuda:0"
print(device)

cuda:0


In [6]:
# def process_config(config) -> DictConfig:
#     """Does whatever operations on the config file
#     """

#     print(OmegaConf.to_yaml(config))
#     print("Working directory : {}".format(os.getcwd()))
#     config.input_size = eval(config.input_size)
#     print("config type: {}".format(type(config)))
#     return config

In [7]:
# from hydra.core.global_hydra import GlobalHydra
# model_dir = 'Runs/28_cingulate_back_to_2mm/Output/t-0.1/n-004_o-4'

# GlobalHydra.instance().clear()
# # Initialize hydra
# initialize(config_path=f"{model_dir}/.hydra")
# # compose from config.yaml, this composes a bunch of defaults in:
# config=compose(config_name="config.yaml")
# config = process_config(config)

In [11]:
# We load the mebeddings
result_dir = '/host/volatile/jc225751/Runs/29_MIDL_2022_clean/Output/t-0.1/n-004_o-4'
val_embeddings_file = f"{result_dir}/train_val_embeddings.pt"
embeddings = torch.load(val_embeddings_file)
print(embeddings.shape)

torch.Size([550, 4])


In [ ]:
# We load the labels
with open(f"{result_dir}/train_filenames.json", 'r') as f:
    labels = json.load(f)
print(labels)

In [12]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(embeddings)
cluster_labels = kmeans.labels_
clusters_centroids = kmeans.cluster_centers_
print(f"clusters centroids = {clusters_centroids.shape}")
print(f"Average silhouette score: {metrics.silhouette_score(embeddings, cluster_labels)}")


clusters centroids = (3, 4)
Average silhouette score: 0.43807780742645264


In [15]:
dstrb_sub = pd.DataFrame(embeddings.numpy(), index=labels, columns=[1, 2, 3, 4])
dstrb_sub['cluster_lab'] = cluster_labels

NameError: name 'labels' is not defined

In [14]:
dstrb_sub.head()

NameError: name 'dstrb_sub' is not defined

In [ ]:
clusters_centroids = kmeans.cluster_centers_
print(f"cluster's centroids coordinates: \n {clusters_centroids}")

In [ ]:
central_1= closest_distance(clusters_centroids[0], dstrb_sub.drop(['cluster_lab'], axis=1))
print(f"Closest subject to centroid of cluster 1 is {central_1}")
central_2 = closest_distance(clusters_centroids[1], dstrb_sub.drop(['cluster_lab'], axis=1))
print(f"Closest subject to centroid of cluster 2 is {central_2}")

In [ ]:
arr = np.array([np.array([dstrb_sub[k][i] for k in dstrb_sub.columns[0:2]]) for i in range(len(dstrb_sub))])

color_dict = {0: 'red', 1:'blue', 2:'green'}
fig, ax = plt.subplots()

for g in np.unique([dstrb_sub.cluster_lab]):
    ix = np.where(dstrb_sub.cluster_lab == g)
    x = [arr[ix][k][0] for k in range(len(ix[0]))]
    y = [arr[ix][k][1] for k in range(len(ix[0]))]
    if g =='benchmark':
        g_lab = 'benchmark asymmetry'
    else:
        g_lab=g
    ax.scatter(x, y, c = color_dict[g], label = g_lab)
    
ax.scatter(clusters_centroids[0][0], clusters_centroids[0][1], color='crimson', marker='X')
ax.scatter(clusters_centroids[1][0], clusters_centroids[1][1], color='navy', marker='X')

plt.xlabel(f'dimension 1', fontsize=14)
plt.ylabel(f'dimension 2', fontsize=14)
plt.legend()
plt.show()

In [ ]:
dstrb_sub.head()

In [ ]:
dstrb_sub.index[0]

In [ ]:
arr = np.array([np.array([dstrb_sub[k][i] for k in dstrb_sub.columns[2:4]]) for i in range(len(dstrb_sub))])

color_dict = {0: 'red', 1:'blue', 2:'green'}
fig, ax = plt.subplots()

for g in np.unique([dstrb_sub.cluster_lab]):
    ix = np.where(dstrb_sub.cluster_lab == g)
    x = [arr[ix][k][0] for k in range(len(ix[0]))]
    y = [arr[ix][k][1] for k in range(len(ix[0]))]
    ax.scatter(x, y, c = color_dict[g], label = g)
    
ax.scatter(clusters_centroids[0][2], clusters_centroids[0][3], color='crimson', marker='X')
ax.scatter(clusters_centroids[1][2], clusters_centroids[1][3], color='navy', marker='X')
# ax.scatter(dstrb_sub[3]['113316'], dstrb_sub[4]['113316'], color='orange')
# ax.scatter(dstrb_sub[3]['144125'], dstrb_sub[4]['144125'], color='forestgreen')
# ax.scatter(dstrb_sub[3]['217126'], dstrb_sub[4]['217126'], color='black')

plt.xlabel(f'dimension 3', fontsize=14)
plt.ylabel(f'dimension 4', fontsize=14)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
for g in np.unique([dstrb_sub.cluster_lab]):
    ix = np.where(dstrb_sub.cluster_lab == g)
    x = [arr[ix][k][0] for k in range(len(ix[0]))]
    y = [arr[ix][k][1] for k in range(len(ix[0]))]
    ax.scatter(x, y, c = color_dict[g], label = g)
    for k, i in zip(range(len(ix[0])), ix[0]):
        ax.annotate(dstrb_sub.index[i], (x[k], y[k]),fontsize=7)
    
ax.scatter(clusters_centroids[0][2], clusters_centroids[0][3], color='crimson', marker='X')
ax.scatter(clusters_centroids[1][2], clusters_centroids[1][3], color='navy', marker='X')
# ax.scatter(dstrb_sub[3]['113316'], dstrb_sub[4]['113316'], color='orange')
# ax.scatter(dstrb_sub[3]['144125'], dstrb_sub[4]['144125'], color='forestgreen')
# ax.scatter(dstrb_sub[3]['217126'], dstrb_sub[4]['217126'], color='black')

plt.xlabel(f'dimension 3', fontsize=14)
plt.ylabel(f'dimension 4', fontsize=14)
plt.legend()
plt.show()

In [ ]:
cluster1 = dstrb_sub[dstrb_sub.cluster_lab==0]
cluster2 = dstrb_sub[dstrb_sub.cluster_lab==1]
assert(len(np.unique(list(cluster1.cluster_lab)))==1)
assert(len(np.unique(list(cluster2.cluster_lab)))==1)

### Creation of buckets dictionnary

In [ ]:
bucket_path = '/neurospin/dico/data/deep_folding/current/crops/CINGULATE/mask/sulcus_based/2mm/centered_combined/hcp/Rbuckets/'
suffix_path = '_normalized.bck'
buckets = {}
for sub in tqdm(list(labels)):
    bucket = aims.read(os.path.join(bucket_path, str(sub) + suffix_path))
    bucket = dtx.convert.bucket_aims_to_ndarray(bucket[0])
    buckets[sub] = bucket

In [ ]:
subjects_c1 = cluster1.index
subjects_c2 = cluster2.index

buckets_c1 = {k: v for k,v in buckets.items() if k in list(subjects_c1)}
buckets_c2 = {k: v for k,v in buckets.items() if k in list(subjects_c2)}

In [ ]:
cld.draw(list(buckets_c1.values())[0])

#### Alignement of the subjects to respective central subject

In [ ]:
aligned_buckets_C1, aligned_rot_C1, aligned_transl_C1 = ma.align_buckets_by_ICP_batch(buckets_c1, central_1)
aligned_buckets_C2, aligned_rot_C2, aligned_transl_C2 = ma.align_buckets_by_ICP_batch(buckets_c2, central_2)

In [ ]:
plt.hist(cluster1[3], alpha=0.5)
plt.hist(cluster2[3], alpha=0.5)
plt.show()

In [ ]:
def subj_count_extreme_coords(isodf, axis, min_coord, max_coord, num_coord=10):
    coord_values = np.linspace(min_coord, max_coord, num_coord)
    step = (coord_values[1] - coord_values[0])
    midpoint_min = coord_values[0] + step/2
    midpoint_max = coord_values[num_coord-1] - step/2
    coord_under = [elt for elt in isodf[axis] if elt<midpoint_min]
    coord_over = [elt for elt in isodf[axis] if elt>midpoint_max]
    num_subj_under = len(coord_under)
    num_subj_over = len(coord_over)
    return num_subj_under, num_subj_over

def get_MA_coords(isodf, axis, num_subj_threshold, num_coord=10):
    min_subj_coord = min(isodf[axis])
    max_subj_coord = max(isodf[axis])
    step = (max_subj_coord - min_subj_coord)/num_coord
    num_subj_under, num_subj_over = subj_count_extreme_coords(isodf, axis, min_subj_coord, max_subj_coord)
    while ((num_subj_under<num_subj_threshold)|(num_subj_over<num_subj_threshold)):
        step = (max_subj_coord - min_subj_coord)/num_coord
        if num_subj_under<num_subj_threshold:
            min_subj_coord = min_subj_coord + step
        if num_subj_over<num_subj_threshold:
            max_subj_coord = max_subj_coord - step
        num_subj_under, num_subj_over = subj_count_extreme_coords(isodf, axis, min_subj_coord, max_subj_coord)
    return np.linspace(min_subj_coord, max_subj_coord, num_coord)

In [ ]:
MA_coords = get_MA_coords(cluster1, 3, num_subj_threshold=14, num_coord=2)
step = step = MA_coords[1]-MA_coords[0]

In [ ]:
MA_coords

In [ ]:
step

In [ ]:
cluster1.index 

In [ ]:
SPAM_vols_c1, shift1 = ma.calc_MA_volumes_batch(MA_coords, aligned_buckets_C1, cluster1, axis_n=3, FWHM=0.5)
SPAM_vols_c1, shift1 = ma.moving_averages_tools.calc_one_MA_volume(aligned_buckets_C1, cluster1, axis_n=3, FWHM=0.5)

In [ ]:
SPAM_vols_c1.items()

In [ ]:
SPAM_meshes = {}

for k, volume in tqdm(list(SPAM_vols_c1.items())[:]):
    SPAM_meshes[k]=dtx.convert.volume_to_mesh(
                    vol=volume,
                    gblur_sigma=0.0,
                    threshold=0.45,
                    deciReductionRate=0,
                    smoothRate=0.25)
    #SPAM_meshes[k] = dtx.mesh.shift_aims_mesh(SPAM_meshes[k], 2*shift1[k])
    
shifted_SPAM_meshes = {}
for dist, mesh in SPAM_meshes.items():
    shifted_SPAM_meshes[str(dist)] = dtx.mesh.shift_aims_mesh_along_axis(mesh, dist, axis=0, scale=30)

In [ ]:
fig = cld.draw(list(SPAM_vols_c1.values())[0], th_min=0.2)
cld.draw(list(SPAM_meshes.values())[0], fig=fig)

In [ ]:
for x, mesh in tqdm(shifted_SPAM_meshes.items()):
    aims.write(mesh, f"MA_{x}_2cluster33.mesh")#### Saving of average crops

### Whole cluster analysis

In [ ]:
clusters_centroids

In [ ]:
np.mean(clusters_centroids,axis=0)

In [ ]:
clusters_centroids = kmeans.cluster_centers_
print(f"cluster's centroids coordinates: \n {clusters_centroids}")
centroid = np.mean(clusters_centroids, axis=0)
print(centroid)

central = closest_distance(centroid, dstrb_sub.drop(['cluster_lab'], axis=1))
print(f"Closest subject to centroid of cluster is {central}")
print(dstrb_sub[3][str(central)], dstrb_sub[4][str(central)])

In [ ]:
aligned_buckets, aligned_rot, aligned_transl = ma.align_buckets_by_ICP_batch(buckets, central)

In [ ]:
MA_coords = get_MA_coords(dstrb_sub, 3, num_subj_threshold=14, num_coord=5)
step = step = MA_coords[1]-MA_coords[0]

In [ ]:
MA_coords

In [ ]:
step

In [ ]:
len(dstrb_sub)

In [ ]:
#SPAM_centers_c1 = [-2.05716]
#SPAM_vols_c1, shift1 = ma.calc_MA_volumes_batch(SPAM_centers_c1, aligned_buckets_C1, cluster1, axis_n=1, FWHM=1)
SPAM_vols, shift = ma.calc_MA_volumes_batch(MA_coords, aligned_buckets, dstrb_sub, axis_n=1, FWHM=1)

In [ ]:
# Draws first volume in the brochette
cld.draw(list(SPAM_vols.values())[0], th_min=0.3)

In [ ]:
# Draws last volume in the brochette
cld.draw(list(SPAM_vols.values())[-1], th_min=0.3)

In [ ]:
SPAM_meshes = {}

for k, volume in tqdm(list(SPAM_vols.items())[:]):
    SPAM_meshes[k]=dtx.convert.volume_to_mesh(
                    vol=volume,
                    gblur_sigma=0,
                    threshold=0.45,
                    deciReductionRate=0,
                    smoothRate=0.4)
    #SPAM_meshes[k] = dtx.mesh.shift_aims_mesh(SPAM_meshes[k], 2*shift1[k])
    
shifted_SPAM_meshes = {}
for dist, mesh in SPAM_meshes.items():
    shifted_SPAM_meshes[str(dist)] = dtx.mesh.shift_aims_mesh_along_axis(mesh, dist, axis=0, scale=20)

In [ ]:
for x, mesh in tqdm(shifted_SPAM_meshes.items()):
    aims.write(mesh, f"MA_{x}_2cluster44.mesh")#### Saving of average crops

In [ ]:
cld.draw(list(SPAM_meshes.values())[0])

In [ ]:
cld.draw(list(SPAM_meshes.values())[4])

In [ ]:
fig = cld.draw(list(SPAM_vols.values())[0], th_min=0.2)
cld.draw(list(SPAM_meshes.values())[0], fig=fig)

In [ ]:
SPAM_centers_c2 = [0]
SPAM_vols_c2, shift2 = ma.calc_MA_volumes_batch(SPAM_centers_c2, aligned_buckets_C2, cluster2, axis_n=3, FWHM=1)

#### Visualization of average crops of both clusters

In [ ]:
fig = cld.draw(shifted_SPAM_meshes)
ma.plot.brochette_layout(fig, "subjects meshes")

In [ ]:

for x, mesh in tqdm(shifted_SPAM_meshes.items()):
    aims.write(mesh, f"MA_{x}_2cluster2.mesh")

# Saves extreme bucket files

In [ ]:
arr = np.array([np.array([dstrb_sub[k][i] for k in dstrb_sub.columns[2:4]]) for i in range(len(dstrb_sub))])

# Selects subjects whose 3rd dimension is higher than 14
ix_sup14 = np.where(arr[:,0] > 14.)
print(ix_sup14)

In [ ]:
for ix in np.nditer(ix_sup14):
    sub = dstrb_sub.index[ix]
    aims.write(dtx.convert.bucket_numpy_to_bucketMap_aims(buckets[sub]), f">14_{sub}.bck")

In [ ]:
# Selects subjects whose 3rd dimension is less than 1
ix_inf1 = np.where(arr[:,0] < 1.)
print(ix_inf1)

In [ ]:
for ix in np.nditer(ix_inf1):
    sub = dstrb_sub.index[ix]
    aims.write(dtx.convert.bucket_numpy_to_bucketMap_aims(buckets[sub]), f"<1_{sub}.bck")